In [ ]:
import requests
from bs4 import BeautifulSoup as bs


In [ ]:
# 실제 사이트에서 html 정보를 받아와서 변수에 저장
data = requests.get('https://finance.naver.com/marketindex/exchangeList.naver').text

In [ ]:
# BeautifulSoup을 이용하여 데이터를 파싱
soup = bs(data, 'html.parser')

In [ ]:
print(soup)

In [ ]:
# find는 html안에서 해당 태그의 첫번째 정보를 출력
tbody = soup.find('tbody')

In [ ]:
# find_all() 해당 태그의 전체 정보를 리스트의 형태로 출력
data_list = soup.find_all('td')

In [ ]:
# data_list를 이용하여 2차원 배열로 새로운 리스트를 생성
# 1차원 리스트 생성
# [[7개씩], [7개씩], [7개씩], [7개씩]] -> 2차원 리스트
_list = []
for i in data_list:
    x = i.get_text()
    x = x.replace("\n", '')
    x = x.strip("\t")
    _list.append(x)
_list

In [ ]:
len(_list)//7

In [ ]:
result = []
for i in range(0, (len(_list)//7), 1):
    x = []
    for j in range(i*7, (i+1)*7, 1):
        x.append(_list[j])
    result.append(x)
result

In [ ]:
result = []
for i in range(len(_list)//7):
    result.append(_list[i*7:(i+1)*7])
result

In [38]:
import pandas as pd

In [40]:
columns = ["통화명", "매매기준율", '현찰(구매)', '현찰(판매)', 
           '송금(보낼때)', '송금(받을때)', '미화환산율']

In [42]:
pd.DataFrame(result, columns=columns).to_csv('test.csv', index=False)

In [44]:
import mod_sql

In [45]:
db = mod_sql.Database('localhost', 'root', '1234', 'ubion_test', 3306)

In [47]:
# DB에 table 생성 (naver)
# 필드는 7개 모두 필드의 타입은 문자열 구성
# 통화명 필드는 primary key
sql = """
    create 
    table 
    `naver`(
        `통화명` varchar(16) primary key, 
        `매매기준율` varchar(16), 
        `현찰(구매)` varchar(16), 
        `현찰(판매)` varchar(16), 
        `송금(보낼때)` varchar(16), 
        `송금(받을때)` varchar(16), 
        `미화환산율` varchar(16)
    )
"""
db.sql_query(sql)

OperationalError: (1050, "Table 'naver' already exists")

In [48]:
# naver라는 이름으로 생성한 테이블에 데이터 삽입
# 데이터를 삽입하는 insert문을 작성
sql = """
    insert 
    into 
    naver 
    values (
        '통화', 
        '1', 
        '2', 
        '3', 
        '4', 
        '5', 
        '6'
    )
"""
db.sql_query(sql)

'Query OK'

In [49]:
sql = """
    insert 
    into 
    naver
    values 
    (%s, %s, %s, %s, %s, %s, %s)
"""

values = [1,2,3,4,5,6,7]

db.sql_query(sql, values)

'Query OK'

In [56]:
sql = """
    delete 
    from 
    `naver`
"""

db.sql_query(sql)

'Query OK'

In [53]:
result[0]

['미국 USD', '1,315.50', '1,338.52', '1,292.48', '1,328.30', '1,302.70', '1.000']

In [57]:
# result변수의 데이터를 naver 테이블 삽입
sql = """
    insert 
    into 
    `naver` 
    values (
        %s, %s, %s, %s, %s, %s, %s
    )
"""

for i in range(len(result)):
    values = result[i]
    db.sql_query(sql, values)

In [ ]:
sql = """
    select * from `naver`
"""
db.sql_query(sql)

In [60]:
# result변수의 데이터를 naver 테이블 삽입
sql = """
    insert 
    into 
    `naver2` 
    (`통화명`, `매매기준율`, `현찰(구매)`, 
    `현찰(판매)`, `송금(보낼때)`, `송금(받을때)`, `미화환산율`)
    values (
        %s, %s, %s, %s, %s, %s, %s
    )
"""

for i in result:
    db.sql_query(sql, i)